In [1]:
"""
This is a skeleton of Assignment 2 for DS5110 Spring 2023.
NOTE you will need to change the .ipynb file name by following the naming convention.
Code should be commented well. 
Feel free to import any missing packages. 
"""
import dask
from dask.distributed import Client
import dask.dataframe as dd
import time
import json
import numpy as np
import pandas as pd

In [2]:
# fill out scheduler IP and port here when creating a Dask client object
client = Client('172.31.84.189:8786') # scheduler IP & necessary port
client = client.restart()
print(client)

<Client: 'tcp://172.31.84.189:8786' processes=4 threads=16, memory=30.66 GiB>


In [3]:
%%time

# Load the .CSV files into the Dask dataframes
questions = dd.read_csv('/home/ubuntu/questions.csv', sample=2**30, assume_missing=True)
question_tags = dd.read_csv('/home/ubuntu/question_tags.csv', sample=2**30, assume_missing=True)

CPU times: user 549 ms, sys: 2.22 s, total: 2.77 s
Wall time: 2.77 s


In [4]:
%%time

# Task 1
# Get the percentage of missing values for all the columns in the questions table and the question_tags table.

percent_missing = questions.isnull().sum()*100 / len(questions) # compute percentage of missing values by column of df
missing_value_df = pd.DataFrame({'column_name': questions.columns,
                                 'percent_missing': percent_missing}) # put output into dataframe

CPU times: user 163 ms, sys: 52.1 ms, total: 215 ms
Wall time: 17.7 s


In [5]:
missing_value_df # display the percent of missing values for each column of questions df

,column_name,percent_missing
0,Id,0.000000
1,CreationDate,0.000000
2,ClosedDate,89.973578
3,DeletionDate,76.962907
4,Score,0.000000
5,OwnerUserId,23.920885
6,AnswerCount,10.114036


In [6]:
%%time

# Task 2
# Get mean, standard deviation, medium, min, and max of the Score column in the questions table.

score_mean = questions['Score'].mean().compute() # compute mean
score_std = questions['Score'].std().compute() # compute std
score_med = questions['Score'].median_approximate().compute() # compute median (approximat works best for dask)
score_min = questions['Score'].min().compute() # compute min
score_max = questions['Score'].max().compute() # compute max

eda = [score_mean, score_std, score_med, score_min, score_max] # put metrics from above into list

eda_df = pd.DataFrame({'metric': ['mean', 'standard_dev', 'median', 'min', 'max'],
                       'value': eda}) # put output into dataframe

CPU times: user 120 ms, sys: 10.7 ms, total: 131 ms
Wall time: 10.6 s


In [7]:
eda_df # eda of score column in questions df displayed as df

,metric,value
0,mean,1.204167
1,standard_dev,15.645736
2,median,1.000000
3,min,-154.000000
4,max,16902.000000


In [8]:
%%time

# Task 3
# Get the top 5 tags that have the highest number of questions (hint: manipulate the question_tags table)

task3 = question_tags.drop_duplicates().compute() # drop duplicate rows (where question Id and tag are the same)
task3['Tag'].value_counts() # sorts in descending order by default - shows top 5 tags with highest no. questions

CPU times: user 8.17 s, sys: 1.48 s, total: 9.65 s
Wall time: 1min 51s


javascript              1649619
java                    1563819
php                     1365600
c#                      1321027
android                 1288193
                         ...   
silverlighttoolkit            1
mvpd                          1
fakesource                    1
okta-password-policy          1
ruby-prolog                   1
Name: Tag, Length: 58254, dtype: int64

The most common tags (the ones with the highest amount of questions associated with them) are as follows:

- `javascript`: 1649619 questions
- `java`: 1563819 questions
- `php`: 1365600 questions
- `c#`: 1321027 questions
- `android`: 1288193 questions

In [9]:
%%time

# Task 4
# Check if there are any dangling references to the question Id field from the questions table to question_tags table.
# Return 1 if there are dangling references; return 0 otherwise.

# if the ClosedDate DNE in questions table for a given Id in question_tags, it is a dangling reference

q_ids = question_tags['Id'].unique().compute().tolist() # get all unique question Ids from tag df
q_dangling = questions[questions['ClosedDate'].isnull()]['Id'].compute().tolist() # find all dangling question Ids 

# find q_ids (that have tags) that are dangling (no ClosedDate) & put into list
dangling_references = list(set(q_ids).intersection(q_dangling)) 

ones = [1]*len(dangling_references) # list of positive dummies (a dangling reference is True)
pos_dummies = pd.DataFrame({'Id': dangling_references,
                            'dangling': ones}) # pos dummies into df

task4 = questions.join(pos_dummies.set_index('Id'), on='Id') # join questions & dangling Id dataframe
task4['dangling'] = task4['dangling'].fillna(0) # fill NaN of dangling col with 0

CPU times: user 14 s, sys: 3.46 s, total: 17.5 s
Wall time: 32.4 s


In [10]:
%%time

# Task 5
# group by OwnerUserId and compute specified methods on given columns of questions df
task5 = questions.groupby(by='OwnerUserId').agg({'Score': 'mean',
                                                 'Id': 'count',
                                                 'AnswerCount': 'sum'}).compute().reset_index()
# rename columns to match requirements (names that make sense for column values)
task5= task5.rename(columns={"Score": "AverageScore", "Id": "NumQuestions", "AnswerCount": "NumAnswers"})

task5.sort_values(by='NumQuestions', ascending=False).head(5) # display new df to top 5 question askers

CPU times: user 558 ms, sys: 226 ms, total: 784 ms
Wall time: 7.64 s


,OwnerUserId,AverageScore,NumQuestions,NumAnswers
363219,875317.0,1.198206,2230,3499.0
9195,39677.0,6.607613,2128,5176.0
1972,4653.0,6.883095,1822,5696.0
9347,34537.0,5.213690,1680,4525.0
41352,179736.0,7.344987,1516,3982.0


We see the top 5 users that asked the most questions shown above.

In [11]:
%%time

# Task 6
# group by Tag and compute specified methods on given columns of a merged df; most bytes stored per worker here
join = questions.merge(question_tags, how="left", on=["Id"]) # join questions and tags on question Id

# group by tag and compute specified methods on given columns of joined df
task6 = join.groupby(by='Tag').agg({'Score': 'mean',
                                    'AnswerCount': 'sum',
                                    'Id': 'count',
                                    'OwnerUserId': 'count'}).compute().reset_index() # compute here to keep kernel from dying
# rename columns to match requirements (names that make sense for output)
task6 = task6.rename(columns={"Score": "AverageScore",
                             "AnswerCount": "NumAnswers",
                             "Id": "NumQuestions",
                             "OwnerUserId": "NumOwners"})

CPU times: user 113 ms, sys: 10.2 ms, total: 123 ms
Wall time: 38.5 s


In [12]:
# top 5 tags assorted by the most questions associated with them
task6.sort_values(by='NumQuestions', ascending=False).head(5)

,Tag,AverageScore,NumAnswers,NumQuestions,NumOwners
57,javascript,1.134249,2252471.0,1649631,1296260
116,java,1.056542,2194354.0,1563820,1189238
0,php,0.375525,1855546.0,1365600,1017270
15,c#,1.340325,1937822.0,1321027,1039994
362,android,1.004450,1420669.0,1288338,936349


The top 5 tags that are associated with the highest number of questions are shown above - `javascript`, `java`, `php`, `c#`, and `android`.

In [13]:
# top 5 tags assorted by the number of answers associated with them
task6.sort_values(by='NumAnswers', ascending=False).head(5)

,Tag,AverageScore,NumAnswers,NumQuestions,NumOwners
57,javascript,1.134249,2252471.0,1649631,1296260
116,java,1.056542,2194354.0,1563820,1189238
15,c#,1.340325,1937822.0,1321027,1039994
0,php,0.375525,1855546.0,1365600,1017270
110,jquery,0.881797,1462029.0,1011324,801493


The top 5 tags associated with the most answers provided are shown above - `javascript`, `java`, `c#`, `php`, and `jquery`. Unsurprisingly, 4 out of 5 of these tags also appear in the top 5 tags associated with the greatest number of questions. We would expect that with more questions, comes more answers, explaining the similarity in output.